## With Conversation

In [32]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import openai

In [ ]:
openai.api_key = 'replace_with_your_api_key'

In [ ]:
# Load the dataset
file_path = './Sample_Energy_Data.csv'
data = pd.read_csv(file_path)
data.head()

,Site Code,Site Name,CA Number,Board Name,Closing Meter Reading,Opening Meter Reading,Unit Consumption,Bill Month,Bill Date,Due Date,...,Opening Meter Reading Date,Closing Meter Reading Date,Maximum Demand,Maximum Demand (KVA),Excess Demand Charges,Power Factor,Power Factor Penalty,Green Tariff (kWh),Digital Payment Benefit,Late Payment Penalty
0,1247_BRANCH,Vashi-Palm Beach,74182524_CODE,MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...,512477.36,503845.52,8632.0,Aug 2024,"Aug 4, 2024","Aug 26, 2024",...,"Jun 30, 2024","Jul 31, 2024",40.0,47.0,3102.0,0.31,44682.38,NaN,490.81,2878.73
1,1247_BRANCH,Vashi-Palm Beach,74182524_CODE,MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...,520875.50,512477.36,8398.0,Sep 2024,"Sep 4, 2024","Sep 24, 2024",...,"Jul 31, 2024","Aug 31, 2024",41.0,46.0,2326.5,0.31,43621.93,NaN,477.82,2799.72
2,1247_BRANCH,Vashi-Palm Beach,74182524_CODE,MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...,503845.52,494273.76,9572.0,Jul 2024,"Jul 6, 2024","Jul 26, 2024",...,"May 31, 2024","Jun 30, 2024",40.0,46.0,2326.5,0.36,44962.08,NaN,NaN,3092.91
3,1247_BRANCH,Vashi-Palm Beach,74182524_CODE,MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...,473400.67,463374.34,10026.0,Apr 2024,"Apr 5, 2024","Apr 25, 2024",...,"Feb 29, 2024","Mar 31, 2024",39.0,46.0,2115.0,0.37,43321.07,NaN,500.00,3034.78
4,1247_BRANCH,Vashi-Palm Beach,74182524_CODE,MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...,494273.76,483506.10,10768.0,Jun 2024,"Jun 9, 2024","Jun 29, 2024",...,"Apr 30, 2024","May 31, 2024",45.0,46.0,2326.5,0.40,46226.21,NaN,500.00,3380.49


In [34]:
# PostgreSQL connection parameters
db_config = {
    'dbname': 'energy_data',
    'user': 'myuser',
    'password': 'mypassword',
    'host': 'localhost',
    'port': 5432
}

# SQL Table Creation Query
def create_table():
    query = """
    CREATE TABLE IF NOT EXISTS energy_data (
        site_code VARCHAR(50),
        site_name VARCHAR(255),
        ca_number VARCHAR(50),
        board_name VARCHAR(255),
        closing_meter_reading FLOAT,
        opening_meter_reading FLOAT,
        unit_consumption FLOAT,
        bill_month VARCHAR(20),
        bill_date DATE,
        due_date DATE,
        amount_before_due_date FLOAT,
        amount_after_due_date FLOAT,
        sanctioned_load FLOAT,
        connected_load FLOAT,
        contract_demand FLOAT,
        kva_40_percent_demand FLOAT,
        tariff VARCHAR(50),
        category VARCHAR(50),
        early_payment_amount FLOAT,
        early_payment_date DATE,
        opening_meter_reading_date DATE,
        closing_meter_reading_date DATE,
        maximum_demand FLOAT,
        maximum_demand_kva FLOAT,
        excess_demand_charges FLOAT,
        power_factor FLOAT,
        power_factor_penalty FLOAT,
        green_tariff_kwh FLOAT,
        digital_payment_benefit FLOAT,
        late_payment_penalty FLOAT,
        PRIMARY KEY (site_code, bill_month)
    );
    """
    return query

# Insert Data into SQL Table
def insert_data(cursor, data):
    for _, row in data.iterrows():
        row = row.where(pd.notnull(row), None)  # Replace NaN/NaT with None
        insert_query = sql.SQL(
            """
            INSERT INTO energy_data (
                site_code, site_name, ca_number, board_name, closing_meter_reading,
                opening_meter_reading, unit_consumption, bill_month, bill_date, due_date,
                amount_before_due_date, amount_after_due_date, sanctioned_load, connected_load,
                contract_demand, kva_40_percent_demand, tariff, category, early_payment_amount,
                early_payment_date, opening_meter_reading_date, closing_meter_reading_date,
                maximum_demand, maximum_demand_kva, excess_demand_charges, power_factor,
                power_factor_penalty, green_tariff_kwh, digital_payment_benefit, late_payment_penalty
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (site_code, bill_month) DO NOTHING;
            """
        )
        cursor.execute(insert_query, tuple(row))

In [35]:
import re

conversation_history = []

def convert_query_to_sql_with_context(user_query):
    """
    Convert a user query to SQL with context from the conversation history.
    """
    # Build the context string from the conversation history
    context = "\n".join(
        [
            f"Query {i+1}: {entry['user_query']}\n"
            f"SQL: {entry['sql_query']}\n"
            f"Results: {entry.get('results', 'No results')}\n"
            for i, entry in enumerate(conversation_history)
        ]
    )
    
    prompt = (
        "You are an SQL expert. Convert the following natural language query into a valid SQL query "
        "that can run on the provided schema. Use the conversation history for context:\n\n"
        "Schema:\n"
        "energy_data(site_code, site_name, ca_number, board_name, closing_meter_reading, opening_meter_reading, "
        "unit_consumption, bill_month, bill_date, due_date, amount_before_due_date, amount_after_due_date, "
        "sanctioned_load, connected_load, contract_demand, kva_40_percent_demand, tariff, category, "
        "early_payment_amount, early_payment_date, opening_meter_reading_date, closing_meter_reading_date, "
        "maximum_demand, maximum_demand_kva, excess_demand_charges, power_factor, power_factor_penalty, "
        "green_tariff_kwh, digital_payment_benefit, late_payment_penalty)\n\n"
        "Conversation History:\n"
        f"{context}\n\n"
        f"User Query: {user_query}\n\n"
        "Make sure to handle subqueries correctly by using `IN` instead of `=` if needed.\n\n"
        "Provide only the SQL query without any explanation or additional text."
    )
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an SQL expert assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        )
        sql_query = response['choices'][0]['message']['content'].strip()
        sql_query = re.sub(r'```(sql)?', '', sql_query).strip()  # Remove Markdown formatting
        return sql_query
    except Exception as e:
        print(f"Error generating SQL query: {e}")
        return None

In [36]:
def format_answer_with_llm(user_query, sql_query, results):
    """
    Use an LLM to format the final response for the user query.

    Args:
        user_query (str): The original user query.
        sql_query (str): The generated SQL query.
        results (list): The results retrieved from the SQL query.

    Returns:
        str: The formatted response from the LLM.
    """
    # Prepare the prompt
    prompt = (
        "You are a helpful assistant. A user has asked a question, and you have been given the query results. "
        "Format the results into a concise, user-friendly response.\n\n"
        f"User Query: {user_query}\n\n"
        f"Generated SQL Query: {sql_query}\n\n"
        "Query Results:\n"
        f"{results}\n\n"
        "Provide a natural language response summarizing the results in a way that is easy to understand for the user."
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7,
        )
        formatted_response = response['choices'][0]['message']['content'].strip()
        return formatted_response

    except Exception as e:
        print(f"Error formatting answer with LLM: {e}")
        return "Unable to format the answer. Please review the raw results."


In [37]:
def execute_user_query_with_context(user_query):
    """
    Execute the user query, track the conversation context, and return formatted results.
    """
    global conversation_history

    # Check for empty query
    if not user_query.strip():
        print("Error: The query is empty. Please enter a valid query.")
        return "Error: Empty query provided."

    
    # Convert the user query to SQL
    sql_query = convert_query_to_sql_with_context(user_query)
    if not sql_query:
        print("Failed to generate SQL query.")
        return

    print(f"Generated SQL Query:\n{sql_query}")
    
    # Execute the query
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        results = cursor.fetchall()
        
        # Save the query, SQL, and results in the conversation history
        conversation_history.append({
            "user_query": user_query,
            "sql_query": sql_query,
            "results": results
        })
        
        print("Query executed successfully. Results:")
        for row in results:
            print(row)
        
        # Format the final answer using the LLM
        final_response = format_answer_with_llm(user_query, sql_query, results)
        print("\nFormatted Response for User:")
        print(final_response)

    except Exception as e:
        print(f"Error executing query: {e}")
        # Save the failed attempt in the conversation history
        conversation_history.append({
            "user_query": user_query,
            "sql_query": sql_query,
            "results": None
        })
    finally:
        if conn:
            cursor.close()
            conn.close()

In [38]:
def main():
    """
    Main function to create the database, insert data, and interact with the user for queries.
    """
    try:
        # Step 1: Connect to the database
        conn = psycopg2.connect(**db_config)
        conn.autocommit = True
        cursor = conn.cursor()

        # Step 2: Create the table
        cursor.execute(create_table())
        print("Table created successfully.")

        # Step 3: Insert the data into the table
        insert_data(cursor, data)
        print("Data inserted successfully.")

        # Step 4: User interaction loop for queries
        while True:
            user_query = input("Enter your query (or type 'exit' to quit): ").strip()
            if user_query.lower() == 'exit':
                print("Exiting. Goodbye!")
                break
            
            # Execute the query and track conversation context
            execute_user_query_with_context(user_query)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        if conn:
            cursor.close()
            conn.close()

In [41]:
if __name__ == "__main__":
    main()

Table created successfully.
Data inserted successfully.


Enter your query (or type 'exit' to quit):  Number of Total sites located in Pune area


Generated SQL Query:
SELECT COUNT(DISTINCT site_code) AS total_sites
FROM energy_data
WHERE site_name LIKE '%Pune%';
Query executed successfully. Results:
(9,)

Formatted Response for User:
There are a total of 9 distinct sites located in the Pune area.


Enter your query (or type 'exit' to quit):  give me the second highest site based on unit consumption for each state.


Generated SQL Query:
SELECT site_name, unit_consumption
FROM energy_data AS ed1
WHERE unit_consumption IN (
    SELECT DISTINCT unit_consumption
    FROM energy_data AS ed2
    WHERE ed2.site_code IN (SELECT DISTINCT site_code FROM energy_data GROUP BY state)
    ORDER BY unit_consumption DESC
    LIMIT 1 OFFSET 1
)
ORDER BY site_name;
Error executing query: column "state" does not exist
LINE 6: ... (SELECT DISTINCT site_code FROM energy_data GROUP BY state)
                                                                 ^



...

Enter your query (or type 'exit' to quit):  Out of these sites, which has the highest power consumption?


Generated SQL Query:
SELECT site_name, unit_consumption
FROM energy_data
WHERE site_code IN (SELECT DISTINCT site_code FROM energy_data WHERE site_name LIKE '%Pune%')
ORDER BY unit_consumption DESC
LIMIT 1;
Query executed successfully. Results:
('Pune-Pride Parmar', 161252.0)

Formatted Response for User:
The site with the highest power consumption among those queried is **Pune-Pride Parmar**, with a total consumption of **161,252 units**.


Enter your query (or type 'exit' to quit):  Can you sort the above sites by their unit consumption


Generated SQL Query:
SELECT site_name, unit_consumption
FROM energy_data
WHERE site_code IN (SELECT DISTINCT site_code FROM energy_data WHERE site_name LIKE '%Pune%')
ORDER BY unit_consumption DESC;
Query executed successfully. Results:
('Pune-Pride Parmar', 161252.0)
('Pune-Pride Parmar', 159508.0)
('Pune-Pride Parmar', 142465.0)
('Pune-Pride Parmar', 139909.0)
('Pune-Pride Parmar', 139147.0)
('Pune-Pride Parmar', 132701.0)
('Pune-Pride Parmar', 128781.0)
('Pune-Pride Parmar', 121523.0)
('Pune-Pride Parmar', 117242.0)
('Camp, Pune', 2075.0)
('Camp, Pune', 2037.0)
('Balewadi, Pune', 2020.0)
('Camp, Pune', 1974.0)
('Balewadi, Pune', 1957.0)
('Tilak Road, Pune, Maharashtra', 1795.0)
('Balewadi, Pune', 1768.0)
('Camp, Pune', 1734.0)
('Camp, Pune', 1687.0)
('Bhosari, Pune - Maharashtra', 1676.0)
('Bhosari, Pune - Maharashtra', 1673.0)
('Tilak Road, Pune, Maharashtra', 1663.0)
('Tilak Road, Pune, Maharashtra', 1640.0)
('Wadgaon Sheri, Pune', 1631.0)
('Balewadi, Pune', 1617.0)
('Camp, Pune',

KeyboardInterrupt: Interrupted by user

In [30]:
conversation_history = []

In [ ]:
if __name__ == "__main__":
    main()

Table created successfully.
Data inserted successfully.


Enter your query (or type 'exit' to quit):  Number of Total sites located in Pune area


Generated SQL Query:
SELECT COUNT(DISTINCT site_code) AS total_sites
FROM energy_data
WHERE site_name LIKE '%Pune%';
Query executed successfully. Results:
(9,)

Formatted Response for User:
There are a total of 9 distinct sites located in the Pune area.


Enter your query (or type 'exit' to quit):  Give me the names of these 9 sites


Generated SQL Query:
SELECT DISTINCT site_name
FROM energy_data
WHERE site_name LIKE '%Pune%';
Query executed successfully. Results:
('Pune-Pride Parmar',)
('Tilak Road, Pune, Maharashtra',)
('Balewadi, Pune',)
('Shivaji Nagar, Pune, Maharashtra',)
('Wadgaon Sheri, Pune',)
('Bhosari, Pune - Maharashtra',)
('Pimpri, Pune',)
('Undri, Pune, Maharashtra',)
('Camp, Pune',)

Formatted Response for User:
Here are the names of the sites located in Pune:

1. Pune-Pride Parmar
2. Tilak Road, Pune, Maharashtra
3. Balewadi, Pune
4. Shivaji Nagar, Pune, Maharashtra
5. Wadgaon Sheri, Pune
6. Bhosari, Pune - Maharashtra
7. Pimpri, Pune
8. Undri, Pune, Maharashtra
9. Camp, Pune

Let me know if you need any more information!


...

Generated SQL Query:
SELECT site_name, AVG(unit_consumption) AS average_consumption
FROM energy_data
WHERE site_name LIKE '%Pune%'
GROUP BY site_name
ORDER BY average_consumption ASC;
Query executed successfully. Results:
('Shivaji Nagar, Pune, Maharashtra', 701.6666666666666)
('Wadgaon Sheri, Pune', 835.5555555555555)
('Undri, Pune, Maharashtra', 965.6363636363636)
('Pimpri, Pune', 1125.2727272727273)
('Balewadi, Pune', 1395.090909090909)
('Tilak Road, Pune, Maharashtra', 1461.3636363636363)
('Bhosari, Pune - Maharashtra', 1463.4)
('Camp, Pune', 1612.6363636363637)
('Pune-Pride Parmar', 138058.66666666666)

Formatted Response for User:
Here are the average unit consumptions for various sites in Pune, sorted in ascending order:

1. **Shivaji Nagar, Pune, Maharashtra**: 701.67
2. **Wadgaon Sheri, Pune**: 835.56
3. **Undri, Pune, Maharashtra**: 965.64
4. **Pimpri, Pune**: 1125.27
5. **Balewadi, Pune**: 1395.09
6. **Tilak Road, Pune, Maharashtra**: 1461.36
7. **Bhosari, Pune - Maharashtra

Enter your query (or type 'exit' to quit):  Calculate the median of unit consumption in the above list


Generated SQL Query:
SELECT site_name, 
       PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_consumption) AS median_consumption
FROM energy_data
WHERE site_name LIKE '%Pune%'
GROUP BY site_name;
Query executed successfully. Results:
('Balewadi, Pune', 1316.0)
('Bhosari, Pune - Maharashtra', 1456.0)
('Camp, Pune', 1609.0)
('Pimpri, Pune', 1203.0)
('Pune-Pride Parmar', 139147.0)
('Shivaji Nagar, Pune, Maharashtra', 691.0)
('Tilak Road, Pune, Maharashtra', 1478.0)
('Undri, Pune, Maharashtra', 1052.0)
('Wadgaon Sheri, Pune', 1079.0)

Formatted Response for User:
Here are the median unit consumption values for various sites in Pune:

- **Balewadi, Pune**: 1,316.0
- **Bhosari, Pune - Maharashtra**: 1,456.0
- **Camp, Pune**: 1,609.0
- **Pimpri, Pune**: 1,203.0
- **Pune-Pride Parmar**: 139,147.0 (notably high)
- **Shivaji Nagar, Pune, Maharashtra**: 691.0
- **Tilak Road, Pune, Maharashtra**: 1,478.0
- **Undri, Pune, Maharashtra**: 1,052.0
- **Wadgaon Sheri, Pune**: 1,079.0

These figures re

Enter your query (or type 'exit' to quit):  Overall median


Generated SQL Query:
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_consumption) AS overall_median_consumption
FROM energy_data
WHERE site_name LIKE '%Pune%';
Query executed successfully. Results:
(1315.5,)

Formatted Response for User:
The overall median unit consumption for sites in Pune is 1315.5.


In [25]:
conversation_history

[{'user_query': 'SELECT site_code, site_name, connected_load\nFROM energy_data\nORDER BY connected_load ASC\nLIMIT 1;',
  'sql_query': 'SELECT site_code, site_name, connected_load\nFROM energy_data\nORDER BY connected_load ASC\nLIMIT 1;',
  'results': [('2352_BRANCH', 'Bhosari, Pune - Maharashtra', 4.0)]}]

## Output verifications

In [24]:
data[data['Connected Load'] == data['Connected Load'].min()]

,Site Code,Site Name,CA Number,Board Name,Closing Meter Reading,Opening Meter Reading,Unit Consumption,Bill Month,Bill Date,Due Date,...,Opening Meter Reading Date,Closing Meter Reading Date,Maximum Demand,Maximum Demand (KVA),Excess Demand Charges,Power Factor,Power Factor Penalty,Green Tariff (kWh),Digital Payment Benefit,Late Payment Penalty
198,2352_BRANCH,"Bhosari, Pune - Maharashtra",170142613488_CODE,MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...,22596.0,21157.4,1439.0,Aug 2024,"Aug 6, 2024","Aug 26, 2024",...,"Jun 30, 2024","Jul 31, 2024",5.0,6.0,0.0,0.535,5219.65,NaN,83.58,495.41


In [40]:
# data.sort_values(by = 'Closing Meter Reading', ascending=False)

In [30]:
import pandas as pd

# Load the sample energy data
file_path = './Sample_Energy_Data.csv'
energy_data = pd.read_csv(file_path)

# Display the first few rows and summary of the data to understand its structure
energy_data.info(), energy_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Site Code                   346 non-null    object 
 1   Site Name                   346 non-null    object 
 2   CA Number                   346 non-null    object 
 3   Board Name                  346 non-null    object 
 4   Closing Meter Reading       335 non-null    float64
 5   Opening Meter Reading       342 non-null    float64
 6   Unit Consumption            345 non-null    float64
 7   Bill Month                  346 non-null    object 
 8   Bill Date                   346 non-null    object 
 9   Due Date                    346 non-null    object 
 10  Amount Before Due Date      346 non-null    float64
 11  Amount After Due Date       345 non-null    float64
 12  Sanctioned Load             310 non-null    float64
 13  Connected Load              270 non

(None,
      Site Code         Site Name      CA Number  \
 0  1247_BRANCH  Vashi-Palm Beach  74182524_CODE   
 1  1247_BRANCH  Vashi-Palm Beach  74182524_CODE   
 2  1247_BRANCH  Vashi-Palm Beach  74182524_CODE   
 3  1247_BRANCH  Vashi-Palm Beach  74182524_CODE   
 4  1247_BRANCH  Vashi-Palm Beach  74182524_CODE   
 
                                           Board Name  Closing Meter Reading  \
 0  MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...              512477.36   
 1  MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...              520875.50   
 2  MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...              503845.52   
 3  MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...              473400.67   
 4  MAHARASHTRA STATE ELECTRICITY DISTRIBUTION COM...              494273.76   
 
    Opening Meter Reading  Unit Consumption Bill Month    Bill Date  \
 0              503845.52            8632.0   Aug 2024  Aug 4, 2024   
 1              512477.36            8398.0   Sep 2024 